In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import requests
import os

In [2]:
# URL of page to be scraped
url = 'http://www.espn.com/nba/salaries'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Examine the results, then determine element that contains sought info
# print(soup.prettify())

In [6]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [7]:
# Create the dropdown list in the form of a list

dropdown = {}
for season in browser.find_by_xpath('//select[@class="tablesm"]//option'):
    if season.value == '#':
        continue
        
    dropdown[season.text] = season.value
#     browser.find_by_xpath(f'//select[@class="tablesm"]//option[@value="{season.value}"]').click()
#     print(season.value)
    
# browser.find_by_xpath(f'//select[@class="tablesm"]//option[@value="{season.value}"]').click()
dropdown

{'2018-2019': '//www.espn.com/nba/salaries',
 '2017-2018': '//www.espn.com/nba/salaries/_/year/2018',
 '2016-2017': '//www.espn.com/nba/salaries/_/year/2017',
 '2015-2016': '//www.espn.com/nba/salaries/_/year/2016',
 '2014-2015': '//www.espn.com/nba/salaries/_/year/2015',
 '2013-2014': '//www.espn.com/nba/salaries/_/year/2014',
 '2012-2013': '//www.espn.com/nba/salaries/_/year/2013',
 '2011-2012': '//www.espn.com/nba/salaries/_/year/2012',
 '2010-2011': '//www.espn.com/nba/salaries/_/year/2011',
 '2009-2010': '//www.espn.com/nba/salaries/_/year/2010',
 '2008-2009': '//www.espn.com/nba/salaries/_/year/2009',
 '2007-2008': '//www.espn.com/nba/salaries/_/year/2008',
 '2006-2007': '//www.espn.com/nba/salaries/_/year/2007',
 '2005-2006': '//www.espn.com/nba/salaries/_/year/2006',
 '2004-2005': '//www.espn.com/nba/salaries/_/year/2005',
 '2003-2004': '//www.espn.com/nba/salaries/_/year/2004',
 '2002-2003': '//www.espn.com/nba/salaries/_/year/2003',
 '2001-2002': '//www.espn.com/nba/salaries/

In [8]:
# Loop through the dropdown list, and then within each element of the list loop through all the "next" fields

df = pd.DataFrame(columns=['RK', 'Name', 'Team', 'Salary', 'Season'])

for season in dropdown:
    
    browser.find_by_xpath(f'//select[@class="tablesm"]//option[@value="{dropdown[season]}"]').first.click()

    a = True
    while a:
        tables = pd.read_html(browser.url)
        page_df = tables[0]
        page_df.columns = ['RK', 'Name', 'Team', 'Salary']
        page_df['Season'] = season
        df = df.append(page_df, ignore_index = True)

        # Click the 'Next' button on each page
        try:
            browser.find_by_xpath('//div[@class="jcarousel-next"]').click()

        except :
            a = False

In [15]:
# Display DataFrame
df.head()

,RK,Name,Team,Salary,Season
0,RK,NAME,TEAM,SALARY,2018-2019
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2018-2019
2,2,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150",2018-2019
3,3,"Chris Paul, PG",Houston Rockets,"$35,654,150",2018-2019
4,4,"Blake Griffin, PF",Detroit Pistons,"$32,088,932",2018-2019


In [16]:
# Save to csv
df.to_csv('Resources\salary_raw.csv')